In [ ]:
from typing import List, Optional
import numpy as np

class Tensor:
    def __init__(self, data=None, requires_grad=False):
        # self.data = self._data_to_numpy(data)
        # Float32 to keep consistency. 
        self.data = np.array(data, dtype=np.float32) #Since numpy already handles type conversion, let's use it to keep code simple.
        self.shape = self.data.shape
        self.size = self.data.size
        self.dtype = self.data.dtype
        self.requires_grad = requires_grad
        self.grad = None
  
    def _data_to_numpy(self, data):
        import numpy as np
        if isinstance(data, list):
            return np.array(data)
    def to_numpy(self):
        return self.data
    def __repr__(self):
        return f"Tensor(data={self.data}, shape={self.shape}, size={self.size}, dtype={self.dtype}, requires_grad={self.requires_grad})"
    
    def __add__(self, other):
        """Custom addition for Tensor objects to create a new Tensor object"""
        if isinstance(other, Tensor):
            return Tensor(self.data + other.data)
            # return self.data + other.data
        elif isinstance(other, (int, float)): #Broadcasting support
            return Tensor(self.data + other)
        else:
            raise ValueError(f"Unsupported operand type(s) for +: '{type(self)}' and '{type(other)}'")
    
    def __sub__(self, other):
        """Custom subtraction for Tensor objects to create a new Tensor object"""
        if isinstance(other, Tensor):
            return Tensor(self.data - other.data)
        elif isinstance(other, (int, float)): #Broadcasting support
            return Tensor(self.data - other)
        else:
            raise ValueError(f"Unsupported operand type(s) for -: '{type(self)}' and '{type(other)}'")
        
    def __mul__(self, other):
        """Custom multiplication for Tensor objects to create a new Tensor object"""
        if isinstance(other, Tensor):
            return Tensor(self.data * other.data)
        elif isinstance(other, (int, float)): #Broadcasting support
            return Tensor(self.data * other)
        else:
            raise ValueError(f"Unsupported operand type(s) for *: '{type(self)}' and '{type(other)}'")

    def __truediv__(self, other):
        if isinstance(other, Tensor):
            return Tensor(self.data / other.data)
        elif isinstance(other, (int, float)): #Broadcasting support
            if other == 0: #Can be floating point zero?
                raise ZeroDivisionError("Cannot divide by zero")
            return Tensor(self.data / other)
        else:
            raise ValueError(f"Unsupported operand type(s) for /: '{type(self)}' and '{type(other)}'")
    # def matmul(self, other):
    #     """For dealing with matrix, numpy uses row first operations i.e., 
    #     if a = [[1,2,3], [4,5,6]] -> a.flat will return flat array in order [1,2,3,4,5,6] instead of [1,4,2,5,3,6]"""
    #     if isinstance(other, Tensor):
    #         # Shape will be dynamic, can be one, two, three and so on...
    #         # Note: matrix multiplication in mathematics is only allowed by d dimemnsions. But tensor = multiple dimensions. Check defination of matrix.
    #         # So, broadcast rules is applied to tensors, to define what can be multiplies similar to a matrix. 
    #         # The order of matrix multiplication matters
            
    #         # Track if we reshaped inputs from 1D
    #         other_was_1d = len(other.data.shape) == 1
    #         self_was_1d = len(self.data.shape) == 1
            
    #         # Work with local copies to avoid modifying the original tensors
    #         self_data = self.data
    #         other_data = other.data
            
    #         if self_was_1d:
    #             self_data = self.data.reshape(1, -1)
    #             print(f"The new shape of self is {self_data.shape}")
    #         if other_was_1d:
    #             other_data = other.data.reshape(-1, 1)
    #             print(f"The new shape of other is {other_data.shape}")

    #         if self_data.shape[1] != other_data.shape[0]:
    #             # raise ValueError("Matrix dimensions do not match")
    #             raise ValueError(f"Inner dimensions must match for matrix multiplication: {self_data.shape[1]} ≠ {other_data.shape[0]}")
    #         new_matrix = np.zeros((self_data.shape[0], other_data.shape[1]))
    #         print(f"Shape of new matrix is: {new_matrix.shape}")

    #         # for val in self.data.flat:
    #         for first_mat_row in range(self_data.shape[0]):
    #             for second_mat_column in range(other_data.shape[1]):
    #                 for common_internal in range(self_data.shape[1]): #or other_data.shape[0]
    #                     new_matrix[first_mat_row][second_mat_column] += self_data[first_mat_row][common_internal]*other_data[common_internal][second_mat_column]
            
    #         # If other was originally 1D, result should be 1D (squeeze the column dimension)
    #         if other_was_1d and new_matrix.shape[1] == 1:
    #             new_matrix = new_matrix.squeeze(axis=1)  # Convert from (n, 1) to (n,)
            
    #         return Tensor(new_matrix)

    
    def reshape(self, *args):
        #Flatten the input into flat numbers
        def flatten_args(args):
            for arg in args:
                yield arg
        args_list = list(flatten_args(args)) 
        if np.prod(np.array(args_list)) != self.size:
            raise ValueError(f"The new shape {args_list} does not match the old shape {self.data.shape}")
        return Tensor(np.reshape(self.data, tuple(args_list)))
        # Note: We can reshape by creating a copy, but to do it wihout creating copy, then we need to interpret how the data is stored in memory: C, F foramt and the stride.
        # First property: the multiplication of the new shape should be always equals to old shape
    
    # Note: The case fails when the input is a vector of shape(3,) so we can convert to (3,1) and then after algorithm, convert back to (3,)
    # Note: The case do not handles higher dimensions than 2d matrix: Have to recursively perofrm matrix multiplication.
    # If we have used np.matmul(x,y ) it automatically have handled all of that!!
    def matmul(self, other):
        """For dealing with matrix, numpy uses row first operations i.e., 
        if a = [[1,2,3], [4,5,6]] -> a.flat will return flat array in order [1,2,3,4,5,6] instead of [1,4,2,5,3,6]"""
        if isinstance(other, Tensor):
            # Shape will be dynamic, can be one, two, three and so on...
            # Note: matrix multiplication in mathematics is only allowed by d dimemnsions. But tensor = multiple dimensions. Check defination of matrix.
            # So, broadcast rules is applied to tensors, to define what can be multiplies similar to a matrix. 
            # The order of matrix multiplication matters
            if len(self.data.shape) == 1:
                self.shape = self.data.reshape(1, -1).shape
                print(f"The new shape of self is {self.shape}")
            if len(other.data.shape) == 1:
                other.shape = other.data.reshape(-1, 1).shape
                print(f"The new shape of other is {other.shape}")

            if self.shape[1] != other.shape[0]:
                raise ValueError("Matrix dimensions do not match")
            new_matrix = np.zeros((self.shape[0], other.shape[1]))
            print(f"Shape of new matrix is: {new_matrix.shape}")

            # for val in self.data.flat:
            for first_mat_row in range(self.shape[0]):
                for second_mat_column in range(other.shape[1]):
                    for common_internal in range(self.shape[1]): #or other.shape[0]
                        new_matrix[first_mat_row][second_mat_column] += self.data[first_mat_row][common_internal]*other.data[common_internal][second_mat_column]
            return Tensor(new_matrix)
            # return Tensor(np.matmul(self.data, other.data))
        if isinstance(other, (int, float)):
            return Tensor(self.data * other)
    # Note: So numpy, pytorch uses BLAS, cuBLAS library to handle matrix multiplication. It is 10000x faster than The current normal approach 
    # It still takes O(n^3) time but with caching, hardware optimised reading (and SIMD utilisation with multiple cores or threads), vectorisation

    
h = Tensor([1,2,3])
z = Tensor([4,5,6])
matrix = Tensor([[1, 2], [3, 4]])  # Shape: (2, 2)
vector = Tensor([10, 20])          # Shape: (2,)
result = matrix + vector           # Broadcasting: (2,2) + (2,) → (2,2)
print(result.data)


[[11. 22.]
 [13. 24.]]


In [52]:
for val in range(1):
    print(val)

0


In [ ]:
vector.shape

(3, 1)

In [ ]:
(1,2)

In [ ]:
import numpy as np
vec = np.array([1,2,3])
# vec.reshape((1,1,-1,1,3,-1)).shape

ValueError: can only specify one unknown dimension

In [ ]:
def reshape2(*args, **kwargs):
    # args = tuple(args)
    print(type(args), args)
reshape2(2,3)

<class 'tuple'> (2, 3)


In [ ]:
np.reshape

In [58]:
# Tensor([[1,2,3],[4,5,6]]).data[1][1]
# a = Tensor([[1,1,2],[1,2,1]])
# b = Tensor([[1,1],[1,2],[2,2]])
matrix = Tensor([[1, 2, 3], [4, 5, 6]])  # 2×3
vector = Tensor([1, 2, 3])  # 3×1 (conceptually)
result = matrix.matmul(vector)
result

The new shape of other is (3, 1)
Shape of new matrix is: (2, 1)


Tensor(data=[[14.]
 [32.]], shape=(2, 1), size=2, dtype=float32, requires_grad=False)

(3,)

In [41]:
# print(vector.data.reshape(1,-1).shape)
# vector2 = Tensor([1,2,3])
import numpy as np
vector2 = np.array([1,2,3])
vector2.reshape(((((1, -1))))).shape


(1, 3)

In [51]:
def flatten_recursive(seq):
    for item in seq:
        if isinstance(item, (list, tuple)):
            yield from flatten_recursive(item)
        else:
            yield item

def flatten_stack(seq):
    stack = [iter(seq)]
    while stack:
        for item in stack[-1]:
            if isinstance(item, (list, tuple)):
                stack.append(iter(item))
                break
            else:
                yield item
        else:
            stack.pop()
import time
start= time.time()
list(flatten_recursive(((((((((((((((((((((((((1,2,3))))))))))))))))))))))))))  
p = time.time() - start
print(p)
start = time.time()
list(flatten_stack(((((((((((((((((((((((((1,2,3))))))))))))))))))))))))))
h = time.time()-start
print(h)
print(h>p, h/p)

2.5272369384765625e-05
2.4080276489257812e-05
False 0.9528301886792453


In [ ]:
def flatten_recursive(seq):
    for item in seq:
        if item ==1:
            yield from flatten_recursive(item)
        # if isinstance(item, (list, tuple)):
        #     yield from flatten_recursive(item)
        else:
            yield item
flatten_recursive((((((((((((((((((((((((((1,2,3))))))))))))))))))))))))))
# for item in (1,1,1,1,1,2,2,3,1,1,1,1,1):
#     print(item) 

1
2
3


In [ ]:
def flatten_recursive(seq):
    for item in seq:
        if isinstance(item, (list, tuple)):
            yield from flatten_recursive(item)
        else:
            yield item
nested = [1, [2, [3, 4], 5], (6, 7, [8, 9, [10]])]
print(list(flatten_recursive(nested)))

In [19]:
vector = Tensor([[1,2,3]])
vector.shape

(1, 3)

In [63]:
def test_unit_tensor_creation():
    """🧪 Test Tensor creation with various data types."""
    print("🧪 Unit Test: Tensor Creation...")

    # Test scalar creation
    scalar = Tensor(5.0)
    assert scalar.data == 5.0
    assert scalar.shape == ()
    assert scalar.size == 1
    assert scalar.requires_grad == False
    assert scalar.grad is None
    assert scalar.dtype == np.float32

    # Test vector creation
    vector = Tensor([1, 2, 3])
    assert np.array_equal(vector.data, np.array([1, 2, 3], dtype=np.float32))
    assert vector.shape == (3,)
    assert vector.size == 3

    # Test matrix creation
    matrix = Tensor([[1, 2], [3, 4]])
    assert np.array_equal(matrix.data, np.array([[1, 2], [3, 4]], dtype=np.float32))
    assert matrix.shape == (2, 2)
    assert matrix.size == 4

    # Test gradient flag (dormant feature)
    grad_tensor = Tensor([1, 2], requires_grad=True)
    assert grad_tensor.requires_grad == True
    assert grad_tensor.grad is None  # Still None until Module 05

    print("✅ Tensor creation works correctly!")


test_unit_tensor_creation()

def test_unit_arithmetic_operations():
    """🧪 Test arithmetic operations with broadcasting."""
    print("🧪 Unit Test: Arithmetic Operations...")

    # Test tensor + tensor
    a = Tensor([1, 2, 3])
    b = Tensor([4, 5, 6])
    result = a + b
    assert np.array_equal(result.data, np.array([5, 7, 9], dtype=np.float32))

    # Test tensor + scalar (very common in ML)
    result = a + 10
    assert np.array_equal(result.data, np.array([11, 12, 13], dtype=np.float32))

    # Test broadcasting with different shapes (matrix + vector)
    matrix = Tensor([[1, 2], [3, 4]])
    vector = Tensor([10, 20])
    result = matrix + vector
    expected = np.array([[11, 22], [13, 24]], dtype=np.float32)
    assert np.array_equal(result.data, expected)

    # Test subtraction (data centering)
    result = b - a
    assert np.array_equal(result.data, np.array([3, 3, 3], dtype=np.float32))

    # Test multiplication (scaling)
    result = a * 2
    assert np.array_equal(result.data, np.array([2, 4, 6], dtype=np.float32))

    # Test division (normalization)
    result = b / 2
    assert np.array_equal(result.data, np.array([2.0, 2.5, 3.0], dtype=np.float32))

    # Test chaining operations (common in ML pipelines)
    normalized = (a - 2) / 2  # Center and scale
    expected = np.array([-0.5, 0.0, 0.5], dtype=np.float32)
    assert np.allclose(normalized.data, expected)

    print("✅ Arithmetic operations work correctly!")


test_unit_arithmetic_operations()

def test_unit_matrix_multiplication():
    """🧪 Test matrix multiplication operations."""
    print("🧪 Unit Test: Matrix Multiplication...")

    # Test 2×2 matrix multiplication (basic case)
    a = Tensor([[1, 2], [3, 4]])  # 2×2
    b = Tensor([[5, 6], [7, 8]])  # 2×2
    result = a.matmul(b)
    # Expected: [[1×5+2×7, 1×6+2×8], [3×5+4×7, 3×6+4×8]] = [[19, 22], [43, 50]]
    expected = np.array([[19, 22], [43, 50]], dtype=np.float32)
    assert np.array_equal(result.data, expected)

    # Test rectangular matrices (common in neural networks)
    c = Tensor([[1, 2, 3], [4, 5, 6]])  # 2×3 (like batch_size=2, features=3)
    d = Tensor([[7, 8], [9, 10], [11, 12]])  # 3×2 (like features=3, outputs=2)
    result = c.matmul(d)
    # Expected: [[1×7+2×9+3×11, 1×8+2×10+3×12], [4×7+5×9+6×11, 4×8+5×10+6×12]]
    expected = np.array([[58, 64], [139, 154]], dtype=np.float32)
    assert np.array_equal(result.data, expected)

    # Test matrix-vector multiplication (common in forward pass)
    matrix = Tensor([[1, 2, 3], [4, 5, 6]])  # 2×3
    vector = Tensor([1, 2, 3])  # 3×1 (conceptually)
    result = matrix.matmul(vector)
    # Expected: [1×1+2×2+3×3, 4×1+5×2+6×3] = [14, 32]
    expected = np.array([14, 32], dtype=np.float32)
    assert np.array_equal(result.data, expected)

    # Test shape validation - should raise clear error
    try:
        incompatible_a = Tensor([[1, 2]])     # 1×2
        incompatible_b = Tensor([[1], [2], [3]])  # 3×1
        incompatible_a.matmul(incompatible_b)  # 1×2 @ 3×1 should fail (2 ≠ 3)
        assert False, "Should have raised ValueError for incompatible shapes"
    except ValueError as e:
        assert "Inner dimensions must match" in str(e)
        assert "2 ≠ 3" in str(e)  # Should show specific dimensions

    print("✅ Matrix multiplication works correctly!")


test_unit_matrix_multiplication()

🧪 Unit Test: Tensor Creation...
✅ Tensor creation works correctly!
🧪 Unit Test: Arithmetic Operations...
✅ Arithmetic operations work correctly!
🧪 Unit Test: Matrix Multiplication...
Shape of new matrix is: (2, 2)
Shape of new matrix is: (2, 2)
The new shape of other is (3, 1)
Shape of new matrix is: (2, 1)
✅ Matrix multiplication works correctly!
